## Example 2 - RBF smoothing and derivatives

This notebook introduces the `TopoMesh` object, which builds the following data structures:

- hill slope
- downhill matrices
- upstream area

in addition to the data structures inherited from `FlatMesh`. These form the necessary structures to propogate information from higher to lower elevations. Derivatives are computed on the mesh to calculate the height field, which may need smoothing to reduce short wavelength features and artefacts.

In this notebook we setup a height field and calculate its derivatives on an unstructued mesh. We smooth the derivatives using the radial-basis function (RBF) smoothing kernel to improve the connectivity of stream pathways.

> Note: The API for the structured mesh is identical

#### Contents

- [Height field](#Height-field)
- [Derivatives and hill slope](#Derivatives-and-hill-slope)
- [Upstream area and stream power](#Upstream-area-and-stream-power)
- [Outflow analysis](#Outflow analysis)

In [ ]:
from quagmire.tools import meshtools
from quagmire import TopoMesh, SurfaceProcessMesh

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# from scipy.ndimage import imread
# from quagmire import tools as meshtools
# from quagmire import SurfaceProcessMesh
%matplotlib inline



In [ ]:
minX, maxX = -5.0, 5.0
minY, maxY = -5.0, 5.0,
dx, dy = 0.005, 0.005

x, y, bmask = meshtools.generate_elliptical_points(minX, maxX, minY, maxY, dx, dy, 10000, 500)

# x, y = meshtools.lloyd_mesh_improvement(x, y, bmask, 5)
DM = meshtools.create_DMPlex_from_points(x, y, bmask)

mesh = SurfaceProcessMesh(DM)

## Height field

We generate a hill slope with multiple channels incised along the boundary. The height and slope fields reside as attributes on the `TopoMesh` instance.

In [ ]:
radius  = np.sqrt((x**2 + y**2))
theta   = np.arctan2(y,x)

height  = np.exp(-0.025*(x**2 + y**2)**2) + 0.25 * (0.2*radius)**4  * np.cos(10.0*theta)**2 ## Less so
height  += 0.5 * (1.0-0.2*radius)
height  += np.random.random(height.size) * 0.01 # random noise


mesh.update_height(height)

# rain proportional to height
rain = height**2
# mesh.update_surface_processes(rain, np.zeros_like(rain))



## Derivatives and hill slopes

**Derivatives** can be evaluated on the mesh using the inbuilt routine in the `stripy` object. It employs automatically selected tension factors to preserve shape properties of the data and avoid overshoot and undershoot associated with steep gradients.

```python
dfdx, dfdy = mesh.derivative_grad(f, nit=10, tol=1e-8):
```

where `nit` and `tol` control the convergence criteria.

---

The **radial-basis function** (RBF) smoothing kernel works by setting up a series of gaussian functions based on the distance $d$ between neighbouring nodes and a `delta` scaling factor, $\Delta$:

$$
W_i = \frac{\exp \left( \frac{d_i}{\Delta} \right)^2}{\sum_{i} \left( \frac{d_i}{\Delta} \right)^2}
$$

`delta` is set to the mean distance between nodes by default, but it may be changed to increase or decrease the _smoothness_:

```python
smoothed_field = mesh.rbf_smoother(field, iterations=1, delta=None)
```

In [ ]:
def smooth_slope(mesh, height, delta=None):
    """ Compute the slope using the RBF smoothing kernel """
    mesh._construct_rbf_weights(delta)
    rbf_height = mesh.rbf_smoother(height)
    rbf_gradx, rbf_grady = mesh.derivative_grad(rbf_height)
    return np.hypot(rbf_gradx, rbf_grady)


# analytic slope



rbf_slope1 = smooth_slope(mesh, mesh.height, delta=0.01)
rbf_slope2 = smooth_slope(mesh, mesh.height, delta=0.03)
rbf_slope3 = smooth_slope(mesh, mesh.height, delta=0.05)

# rbf_height = mesh.rbf_smoother(mesh.height)
# rbf_gradx, rbf_grady = mesh.derivative_grad(rbf_height)
# rbf_slope1 = np.hypot(rbf_gradx, rbf_grady)

In [ ]:
import lavavu

points = np.column_stack([mesh.tri.points, height])

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

tri1 = lv.triangles("triangles")
tri1.vertices(points)
tri1.indices(mesh.tri.simplices)
tri1.values(mesh.slope, "slope")
tri1.values(rbf_slope1, "smooth_slope")
tri1.colourmap("#990000 #FFFFFF #000099")
tri1.colourbar()

lv.control.Panel()
lv.control.ObjectList()
tri1.control.List(["slope", "smooth_slope"], property="colourby", value="orginal", command="redraw")
lv.control.show()

## Upstream area and stream power

Integrating information upstream is the cornerstone of stream power laws that govern landscape evolution. This is computed by multiple $\mathbf{D} \cdot \mathbf{A}_{\mathrm{upstream}}$ evaluations to accumulate the area downstream node-by-node on the mesh. This is handled by the `cumulative_flow(vector)` routine.

In [derivatives and hill slopes](#Derivatives-and-hill-slopes) we smoothed the entire landscape, however we can also target the RBF kernel to smooth just the streams:

```python
streamwise_smoothing(data, its, centre_weight=0.75)
```

where `its` indicates the number of iterations to smooth the field stream-wise. Increasing `its` smooths the field further afield upstream and downstream.

In [ ]:
def compute_stream_power(mesh, m=1, n=1):
    """
    Stream power law (q_s)
    """
    rainflux = mesh.rainfall_pattern
    rainfall = mesh.area * rainflux
    cumulative_rain = mesh.cumulative_flow(rainfall)

    stream_power = cumulative_rain**m * mesh.slope**n
    return stream_power


mesh.rainfall_pattern = rain

# mesh._construct_rbf_weights(delta=0.03)
rbf_height = mesh.rbf_smoother(mesh.height)
rbf_gradx, rbf_grady = mesh.derivative_grad(rbf_height)
rbf_slope = np.hypot(rbf_gradx, rbf_grady)

stream_power = compute_stream_power(mesh, m=1, n=1)
print("max stream power = {:.4f}, mean stream power = {:.4f}".format(stream_power.max(), stream_power.mean()))

# stream-wise smoothing
stream_power = mesh.streamwise_smoothing(stream_power, 3, 0.5)
print("max stream power = {:.4f}, mean stream power = {:.4f}".format(stream_power.max(), stream_power.mean()))

# zero stream power at the boundary nodes
stream_power[~mesh.bmask] = 0.0
print("max stream power = {:.4f}, mean stream power = {:.4f}".format(stream_power.max(), stream_power.mean()))


cumulative_rain = mesh.cumulative_flow(mesh.rainfall_pattern * mesh.area)

In [ ]:
lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

tri1 = lv.triangles("triangles")
tri1.vertices(points)
tri1.indices(mesh.tri.simplices)
tri1.values(mesh.slope, "slope")
tri1.values(rbf_slope1, "rain")
tri1.values(stream_power, "stream_power")
tri1.colourmap("drywet")
tri1.colourbar()

lv.control.Panel()
lv.control.ObjectList()
tri1.control.List(["slope", "rain", "stream_power"], property="colourby", value="orginal", command="redraw")
lv.control.show()

## Outflow analysis

The topography we have defined has multiple outflow points, which, depending on the connectivity of the mesh, should be relatively equal.

In [ ]:
outflow_nodes = mesh.identify_outflow_points()

print("{} outflow nodes:".format(len(outflow_nodes)))
print(outflow_nodes)


cum_rain = cumulative_rain[outflow_nodes]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot bar graph of cumulative rain for each outflow point

fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(111, xlabel='outflow node', ylabel='cumulative rain')
ax1.bar(list(range(0,len(cum_rain))), height=cum_rain)
plt.show()


In [ ]:
r = mesh.neighbour_cloud_distances[:,:5].mean(axis=1)
m = mesh.neighbour_cloud_distances[:,1] 
w = mesh.gaussian_dist_w[:,0]

a = w / r

a.max()/a.min()

The downhill matrices are introduced in the next example, [Ex3-Multiple-downhill-pathways](./Ex3-Multiple-downhill-pathways.ipynb)